<a href="https://colab.research.google.com/github/lauramoraes/Analise_de_Clusters/blob/master/2%20-%20Medidas%20de%20Similaridade%20e%20Dist%C3%A2ncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similaridade e distância

Utilizamos medidas de similaridade (ou distância) para quantificar o quanto dois objetos se parecem. Essa informação é útil em diversas aplicações, como:

1. Sistemas de recomendação
2. Agrupamentos
3. Classificação e regressão
4. Detecção de anomalias/outliers
5. Descoberta de casos parecidos (diagnóstico médico, precedentes legais)


### Importando bibliotecas a serem usadas

In [11]:
install.packages("proxy")
install.packages("gdata")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


In [0]:
suppressMessages(suppressWarnings(library(tidyverse)))
suppressMessages(suppressWarnings(library(proxy)))
suppressMessages(suppressWarnings(library(gdata)))

# Whisky Analytics

Foram criadas caraterísticas numéricas que resumem as informações das notas de degustação. Foram definidos cinco atributos gerais de uísquem cada um com muitos valores:

<img src="https://drive.google.com/uc?export=view&id=1mN_UmQeD43_8WfA9hiIRANXLnzAcgKIM">


É importante notar que essas categorias de valores **não** são mutuamente exclusivas por exemplo, o paladar de Aberlour é descrito como médio, íntegro, suave, completo e macio). Em geral, qualquer um dos valores pode ocorrer ao mesmo tempo (embora alguns deles, como a cor ser clara e esfumaçada nunca acontece), mas como podem coocorrer, cada valor de cada variável foi codificado como uma característica separada.

### Carregando os dados salvos anteriormente

In [13]:
list.files()

[1] "data_cleaned.Rdata" "data_names.Rdata"   "data_onehot.Rdata" 
[4] "sample_data"

In [14]:
vars = load("data_names.Rdata")
vars
vars = load("data_cleaned.Rdata")
vars
vars = load("data_onehot.Rdata")
vars
head(data_cleaned)

[1] "data_names"

[1] "data_cleaned"

[1] "data_onehot"

wyne,yellow,v.pale,pale,p.gold,gold,o.gold,f.gold,bronze,p.amber,⋯,DISTRICT,islay,midland,spey,east,west,north,lowland,campbell,islands
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,1,0,0,0,0,0,0,0,0,⋯,MIDLAND,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,SPEY,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,⋯,SOUTH,1,0,0,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,⋯,SPEY,0,0,1,0,0,0,0,0,0
0,0,0,0,1,0,0,0,0,0,⋯,WEST,0,0,0,0,0,0,1,0,0
0,0,1,0,0,0,0,0,0,0,⋯,SPEY,0,0,1,0,0,0,0,0,0


In [15]:
data_names

NAME
<fct>
Aberfeldy
Aberlour
Ardberg
Ardmore
Auchentoshan
Aultmore
Balblair
Balmenach
Balvenie



### Calculando distâncias

In [16]:
# Adicionando coluna de nome
rownames(data_cleaned) <- data_names$NAME

# Selecionando alguns Whiskies
names_list <- c("Bunnahabhain", "Glenglassaugh", "Tullibardine", "Ardberg", "Bruichladdich", "Glenmorangie")
data_selected <- data_cleaned[names_list,]

# Selecionando algumas propriedades
color <- c("gold", "pale", "sherry", "p.gold")
body <- c('soft','med', 'full', 'round', 'smooth', 'light', 'firm', 'oily')
palate <- c('full.1', 'dry', 'sherry.1', 'big', 'light.1', 'smooth.1', 'clean', 'fruit', 'grass', 'smoke', 'sweet', 'spice', 'oil', 'salt', 'arome')
nose <- c("AROMA", 'PEAT', 'SWEET', 'LIGHT', 'FRESH', 'DRY', 'FRUIT', 'GRASS', 'SEA', 'SHERRY', 'SPICY', 'RICH')
finish <- c('full.2', 'dry.1', 'warm', 'big.1', 'light.2', 'smooth.2', 'clean.1', 'fruit.1', 'grass.1', 'smoke.1', 'sweet.1', 'spice.1', 'oil.1', 'salt.1', 'arome.1', 'ling', 'long', 'very', 'quick')
attributes <- c(color, body, palate, nose, finish)
data_selected <- data_selected[,attributes]
data_selected

,gold,pale,sherry,p.gold,soft,med,full,round,smooth,light,⋯,smoke.1,sweet.1,spice.1,oil.1,salt.1,arome.1,ling,long,very,quick
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Bunnahabhain,1,0,0,0,0,1,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0
Glenglassaugh,1,0,0,0,0,0,0,0,1,1,⋯,0,0,0,0,0,0,0,0,0,0
Tullibardine,1,0,0,0,0,1,0,0,1,0,⋯,0,1,0,0,0,1,0,0,0,0
Ardberg,0,0,1,0,0,1,1,0,0,1,⋯,0,0,0,0,1,0,0,0,0,0
Bruichladdich,0,1,0,0,0,0,0,0,1,1,⋯,0,0,0,0,0,0,0,0,0,0
Glenmorangie,0,0,0,1,0,1,0,0,0,1,⋯,0,0,0,0,0,0,0,1,0,0


Existem diversos tipos de distância. Quando os atributos são heterogêneos e categóricos, temos algumas alternativas:
1. Categorizá-los em números (como fizemos anteriormente)
2. Usar uma distância apropriada para dados categóricos (como a de Jaccard, por exemplo, utilizada nesse exemplo do Whisky).

Exemplos de distância são:
1. Distância Euclidiana: normalmente a mais utilizada. É geral, intuitiva e computacionalmente muito rápida.
1. Distância de Manhattan
2. Distância de coseno: útil quando se quer ignorar diferenças de escala, como o tamanho dos textos.
3. Distância de Jaccard: Trata dois objetos como conjuntos de características. Apropriado para problemas em que a posse de uma característica comum é importante, mas a ausência não.


In [17]:
dist(data_selected, method="Jaccard", by_rows=TRUE)

              Bunnahabhain Glenglassaugh Tullibardine   Ardberg Bruichladdich
Glenglassaugh    0.6428571                                                   
Tullibardine     0.6470588     0.7058824                                     
Ardberg          0.6666667     0.8125000    0.8500000                        
Bruichladdich    0.6666667     0.7333333    0.7894737 0.8235294              
Glenmorangie     0.6666667     0.7222222    0.8260870 0.8000000     0.8571429

In [18]:
pr_DB$get_entry("fJaccard")

      names fJaccard, fuzzy_Jaccard
        FUN R_fuzzy_dist
   distance TRUE
     PREFUN pr_fJaccard_prefun
    POSTFUN NA
    convert pr_dist2simil
       type metric
       loop FALSE
      C_FUN TRUE
    PACKAGE proxy
       abcd FALSE
    formula sum_i (min{x_i, y_i} / max{x_i, y_i})
  reference Miyamoto S. (1990). Fuzzy sets in information retrieval and
            cluster analysis, Kluwer Academic Publishers, Dordrecht.
description The fuzzy Jaccard dissimilarity (C implementation).

In [19]:
pr_dist2simil(dist(data_selected, method="fJaccard", by_rows=TRUE))

              Bunnahabhain Glenglassaugh Tullibardine   Ardberg Bruichladdich
Glenglassaugh    0.6086957                                                   
Tullibardine     0.6071429     0.5862069                                     
Ardberg          0.6000000     0.5517241    0.5405405                        
Bruichladdich    0.6000000     0.5769231    0.5588235 0.5483871              
Glenmorangie     0.6000000     0.5806452    0.5476190 0.5555556     0.5384615